### Run this notebook on Anaconda to access Azure.
#### Azure couldn't be accessed on QBraid due to Timeout error.

In [1]:
from pytket import Circuit, Qubit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from pytket.passes import FullPeepholeOptimise
from pytket.circuit import OpType

In [2]:
#pip install qiskit

In [3]:
from pytket.qasm import circuit_to_qasm, circuit_from_qasm
                          
qasmfile = 'H2JordanWignerMapper.qasm'
qch2 = circuit_from_qasm(qasmfile)

In [4]:
print('Circuit depth of H2 Jordan Wigner:',  qch2.depth())
print('Circuit CNOT depth of H2 Jordan Wigner:',  qch2.depth_by_type(OpType.CX))
print('Circuit CX depth of H2 Jordan Wigner:', qch2.n_gates_of_type(OpType.CX))

Circuit depth of H2 Jordan Wigner: 83
Circuit CNOT depth of H2 Jordan Wigner: 52
Circuit CX depth of H2 Jordan Wigner: 56


In [5]:
from pytket.circuit.display import render_circuit_jupyter
q = QuantumRegister(3)
qc = QuantumCircuit(q)
from math import pi


In [6]:
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket.circuit import Circuit, OpType


In [7]:
from pytket.extensions.qiskit import AerBackend


In [8]:
from pytket.extensions.qiskit import IBMQBackend
from pytket import Circuit
from pytket.extensions.qiskit import IBMQEmulatorBackend
from qiskit import IBMQ

### Run the H2 Jordan Wigner in Quantinuum and IonQ via AZURE

In [9]:
from pytket.extensions.qsharp import AzureBackend

Preparing Q# environment...


#### IonQ

In [10]:
backend_ionQ = AzureBackend("ionq.simulator",resourceId="/subscriptions/bedaf24b-f5ad-4e96-9920-85897f941499/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rina25",location="westus")
#backend_ionQ = AzureBackend("ionq.qpu",resourceId="/subscriptions/bedaf24b-f5ad-4e96-9920-85897f941499/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rina25",location="westus")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace rina25 in location westus.
Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


In [11]:
qasmfile = 'H2JordanWignerMapper.qasm'
qch2 = circuit_from_qasm(qasmfile)

### H2 Jordan Wigner qch2
render_circuit_jupyter(qch2)

Properties of the H2 Jordan Wigner without any optimizations:

In [12]:
print('Depths:')
print('Circuit depth of H2 Jordan Wigner (no optimization):', qch2.depth())
print('CX depth of H2 Jordan Wigner (no optimization):', qch2.depth_by_type(OpType.CX))
print('CX count of H2 Jordan Wigner (no optimization):', qch2.n_gates_of_type(OpType.CX))


Depths:
Circuit depth of H2 Jordan Wigner (no optimization): 83
CX depth of H2 Jordan Wigner (no optimization): 52
CX count of H2 Jordan Wigner (no optimization): 56


In [13]:
backend_ionQ.valid_circuit(qch2)

False

Since the result is False, we need to compile the H2 Jordan Wigner circuit to IonQ backend.

In [14]:

qasmfile = 'H2JordanWignerMapper.qasm'
qch2 = circuit_from_qasm(qasmfile)
### H2 Jordan Wigner
compiled_qch2IonQ = backend_ionQ.get_compiled_circuit(qch2,2)

print('Depths:')
print('Circuit depth of compiled H2 Jordan Wigner on IonQ (Azure):', compiled_qch2IonQ.depth())
print('Circuit CNOT depth of compiled H2 Jordan Wigner on IonQ (Azure):', compiled_qch2IonQ.depth_by_type(OpType.CX))
####print('Circuit U3 depth of compiled H2 Jordan Wigner on IonQ (Azure):', compiled_qch2IonQ.depth_by_type(OpType.U3))

print('\nGates count:')
print('Circuit CNOT count of compiled H2 Jordan Wigner on IonQ (Azure):',compiled_qch2IonQ.n_gates_of_type(OpType.CX))
render_circuit_jupyter(compiled_qch2IonQ)



Depths:
Circuit depth of compiled H2 Jordan Wigner on IonQ (Azure): 99
Circuit CNOT depth of compiled H2 Jordan Wigner on IonQ (Azure): 33

Gates count:
Circuit CNOT count of compiled H2 Jordan Wigner on IonQ (Azure): 34


If we run it on the real IonQ qpu device, we won't the correct result "1010" with 100% probability, as in the simulator. \
To measure the error, we count how many correct answers we get for 100 shots.\
Correct result was counted 42 times. \
Correct result rate in qpu: 42%
Correct result rate in simulator: 100%

In [15]:
compiled_qch2IonQ.measure_all()
quantum_handleqch2Ionq = backend_ionQ.process_circuit(compiled_qch2IonQ, n_shots=100)
counts = backend_ionQ.get_result(quantum_handleqch2Ionq).get_counts()
print(counts)

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator
Submitting TketCircuit to target ionq.simulator...
Job successfully submitted.
   Job name: TketCircuit
   Job ID: 5e7d531d-4f39-494e-9ffb-bff61e08ac77
Counter({(1, 0, 1, 0): 100})


#### Optimize with FullPeephole

In [16]:
FullPeepholeOptimise().apply(compiled_qch2IonQ) #OPTIMIZATION

print('IonQ (Azure) after FullPeephole:')

print('\nCircuit depth after FPO:', compiled_qch2IonQ.depth())
print("CX depth after FPO", compiled_qch2IonQ.depth_by_type(OpType.CX))
print("CX count after FPO", compiled_qch2IonQ.n_gates_of_type(OpType.CX))

IonQ (Azure) after FullPeephole:

Circuit depth after FPO: 66
CX depth after FPO 33
CX count after FPO 34


In [17]:
#backend_ionQ.valid_circuit(compiled_IonQ)
compiled_qch2IonQ = backend_ionQ.get_compiled_circuit(compiled_qch2IonQ,2)

backend_ionQ.valid_circuit(compiled_qch2IonQ)

True

Repeat FullPeephole again for further optimization.

In [18]:
FullPeepholeOptimise().apply(compiled_qch2IonQ) #OPTIMIZATION

print('IonQ (Azure) after FullPeephole:')

print('\nCircuit depth after FPO:', compiled_qch2IonQ.depth())
print("CX depth after FPO", compiled_qch2IonQ.depth_by_type(OpType.CX))
print("CX count after FPO", compiled_qch2IonQ.n_gates_of_type(OpType.CX))

IonQ (Azure) after FullPeephole:

Circuit depth after FPO: 46
CX depth after FPO 25
CX count after FPO 26


No change after a second FullPeephole

In [19]:
compiled_qch2IonQ = backend_ionQ.get_compiled_circuit(compiled_qch2IonQ,2)
backend_ionQ.valid_circuit(compiled_qch2IonQ)

True

In [20]:
#print('\nCircuit depth after FPO:', compiled_qch2IonQ.depth())
#print("CX depth after FPO", compiled_qch2IonQ.depth_by_type(OpType.CX))
#print("CX count after FPO", compiled_qch2IonQ.n_gates_of_type(OpType.CX))

#### Job submission to Quantinuum

In [21]:
backend_Q = AzureBackend("quantinuum.hqs-lt-s1-sim",resourceId="/subscriptions/bedaf24b-f5ad-4e96-9920-85897f941499/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rina25",location="westus")
#backend_Q = AzureBackend("quantinuum.hqs-lt",resourceId="/subscriptions/bedaf24b-f5ad-4e96-9920-85897f941499/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rina25",location="westus")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace rina25 in location westus.
Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.hqs-lt-s1-sim


In [22]:

backend_Q.valid_circuit(qch2)

False

In [23]:
qasmfile = 'H2JordanWignerMapper.qasm'
qch2 = circuit_from_qasm(qasmfile)

### H2 Jordan Wigner
compiled_qch2Q = backend_Q.get_compiled_circuit(qch2,2)
render_circuit_jupyter(compiled_qch2Q)

print('Depths:')
print('Circuit depth of compiled H2 Jordan Wigner on Quantinuum (Azure):',compiled_qch2Q.depth())
print('Circuit CNOT depth of compiled H2 Jordan Wigner on Quantinuum (Azure):',compiled_qch2Q.depth_by_type(OpType.CX))
#print('Circuit U3 depth of compiled H2 Jordan Wigner on Quantinuum (Azure):',compiled_qch2Q.depth_by_type(OpType.U3))

print('\nGates count:')
print('Circuit CNOT count of compiled H2 Jordan Wigner on Quantinuum (Azure):',compiled_qch2Q.n_gates_of_type(OpType.CX))

Depths:
Circuit depth of compiled H2 Jordan Wigner on Quantinuum (Azure): 99
Circuit CNOT depth of compiled H2 Jordan Wigner on Quantinuum (Azure): 33

Gates count:
Circuit CNOT count of compiled H2 Jordan Wigner on Quantinuum (Azure): 34


In [24]:
backend_Q.valid_circuit(compiled_qch2Q)

True

In [25]:
compiled_qch2Q.measure_all()

#It may take too long
quantum_handle = backend_Q.process_circuit(compiled_qch2Q, n_shots=10)
print(backend_Q.circuit_status(quantum_handle))
quantum_counts = backend_Q.get_result(quantum_handle).get_counts()
print(quantum_counts)



Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.hqs-lt-s1-sim
Submitting TketCircuit to target quantinuum.hqs-lt-s1-sim...
Job successfully submitted.
   Job name: TketCircuit
   Job ID: 88ecd1a4-9894-4bcf-a829-39b587365070
CircuitStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message="{'id': '88ecd1a4-9894-4bcf-a829-39b587365070', 'name': 'TketCircuit', 'status': 'Waiting', 'uri': 'https://portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/bedaf24b-f5ad-4e96-9920-85897f941499/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rina25/job_management?microsoft_azure_quantum_jobid=88ecd1a4-9894-4bcf-a829-39b587365070', 'provider': 'quantinuum', 'target': 'quantinuum.hqs-lt-s1-sim', 'creation_time': '2022-08-24T00:09:11.3610885+00:00', 'begin_execution_time': None, 'end_execution_time': None, 'cost_estimate': ''}", error_detail=None, completed_time=None, queued_time=None, submit

##### Comment: The Quantinuum simulator returned 90% correct final (1 0 1 0) in our last measurement, but this value can vary, e.g. 70%.
A warning about the code above, sometimes it generates an error.

In [26]:
#Further optimization
FullPeepholeOptimise().apply(compiled_qch2Q) #OPTIMIZATION

print('Quantinuum (Azure) after first FullPeephole:')

print('\nCircuit depth after FPO:', compiled_qch2Q.depth())
print("CX depth after FPO", compiled_qch2Q.depth_by_type(OpType.CX))
print("CX count after FPO", compiled_qch2Q.n_gates_of_type(OpType.CX))

Quantinuum (Azure) after first FullPeephole:

Circuit depth after FPO: 66
CX depth after FPO 33
CX count after FPO 34


In [27]:
compiled_qch2Q = backend_Q.get_compiled_circuit(compiled_qch2Q,2)
backend_Q.valid_circuit(compiled_qch2Q)

True

In [28]:
FullPeepholeOptimise().apply(compiled_qch2Q) #OPTIMIZATION

print('Quantinuum (Azure) after second FullPeephole:')

print('\nCircuit depth after FPO:', compiled_qch2Q.depth())
print("CX depth after FPO", compiled_qch2Q.depth_by_type(OpType.CX))
print("CX count after FPO", compiled_qch2Q.n_gates_of_type(OpType.CX))

Quantinuum (Azure) after second FullPeephole:

Circuit depth after FPO: 46
CX depth after FPO 25
CX count after FPO 26


In [29]:
compiled_qch2Q = backend_Q.get_compiled_circuit(compiled_qch2Q,2)
backend_Q.valid_circuit(compiled_qch2Q)

True

In [30]:
print('Quantinuum (Azure) after FullPeephole:')

print('\nCircuit depth after FPO:', compiled_qch2Q.depth())
print("CX depth after FPO", compiled_qch2Q.depth_by_type(OpType.CX))
print("CX count after FPO", compiled_qch2Q.n_gates_of_type(OpType.CX))

Quantinuum (Azure) after FullPeephole:

Circuit depth after FPO: 63
CX depth after FPO 21
CX count after FPO 22


#### Comment: a third application of FPO increases the circuit depth and decreases slightly the CX depth and count.

In [31]:
compiled_qch2Q = backend_Q.get_compiled_circuit(compiled_qch2Q,2)
backend_Q.valid_circuit(compiled_qch2Q)

True

In [32]:
### H2 Jordan Wigner results
#compiled_qch2Q.measure_all()

#quantum_handle = backend_Q.process_circuit(compiled_qch2Q, n_shots=100)
#print(backend_Q.circuit_status(quantum_handle))
#quantum_counts = backend_Q.get_result(quantum_handle).get_counts()
#print(quantum_counts)

No need for more FullPeephole runs because it doesn't improve the optimisation results any further

In [33]:
backend_ionQ.required_predicates #IonQ Azure

[NoMidMeasurePredicate,
 NoSymbolsPredicate,
 NoClassicalControlPredicate,
 NoFastFeedforwardPredicate,
 GateSetPredicate:{ Ry Rz PauliExpBox Measure CX X CnX Z CCX S SWAP T noop Y H Rx }]

In [34]:
backend_Q.required_predicates #Quantinuum Azure

[NoMidMeasurePredicate,
 NoSymbolsPredicate,
 NoClassicalControlPredicate,
 NoFastFeedforwardPredicate,
 GateSetPredicate:{ Ry Rz PauliExpBox Measure CX X CnX Z CCX S SWAP T noop Y H Rx }]